In [7]:
!pip install flask
!pip install flask_pymongo

   ---------------------------------------- 0.0/468.2 kB ? eta -:--:--
   -- ------------------------------------- 30.7/468.2 kB 1.3 MB/s eta 0:00:01
   ---------- ----------------------------- 122.9/468.2 kB 1.2 MB/s eta 0:00:01
   -------------------- ------------------- 235.5/468.2 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------  460.8/468.2 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 468.2/468.2 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/300.4 kB ? eta -:--:--
   --------------------------------------  297.0/300.4 kB 17.9 MB/s eta 0:00:01
   ---------------------------------------- 300.4/300.4 kB 4.6 MB/s eta 0:00:00


In [13]:
from flask import Flask
from flask_pymongo import PyMongo
from bson.objectid import ObjectId
from bson.json_util import dumps

In [17]:
from flask import jsonify,request

In [29]:
app=Flask(__name__)
app.secret_key='secretkey'
app.config['MONGO_URI']= "mongodb://localhost:27017/Ecommerce"
mongo=PyMongo(app)

@app.route('/add',methods=['POST'])
def add_products():
    _json=request.json
    _product_name=_json['product_name']
    _product_id=_json['product_id']

    if _product_name and _product_id and request.method=='POST':
        id=mongo.db.Products.insert_one({'product_name':_product_name,'product_id':_product_id})
        resp=jsonify("Products added successfully")
        resp.status_code=200
        return resp
    else:
        return not_found()
@app.route('/products',methods=['GET'])
def products():
    products=mongo.db.Products.find()
    resp=dumps(products)
    return resp

@app.route('/products/<id>')
def product(id):
    product=mongo.db.Products.find_one({"_id":ObjectId(id)})
    resp=dumps(product)
    return resp

@app.route('/delete/<id>',methods=['DELETE'])
def delete_product(id):
    mongo.db.Products.delete_one({'_id':ObjectId(id)}) 
    resp=jsonify("Products deleted successfully")
    resp.status_code=200
    return resp

@app.route('/update/<id>',methods=['PUT'])
def product_update(id):
    _id=id
    _json=request.json
    _product_name=_json['product_name']
    _product_id=_json['product_id']
    if _product_name and _product_id and request.method=='PUT':
        mongo.db.Products.update_one({'_id':ObjectId(id['$oid']) if '$oid' in _id else ObjectId(_id)},{"$set":{"product_name":_product_name,"product_id":_product_id}})
        resp=jsonify("Product updated successfully")
        resp.status_code=200
        return resp
    else:
        return not_found()
    
@app.errorhandler(404)
def not_found(error=True):
    message={
        'status':404,
        'message':'Not Found'+request.url
    }
    resp=jsonify(message)
    resp.status_code=404
    return resp



if __name__=="__main__":
    app.run(debug=True,use_reloader=False)
    

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [27/Oct/2023 01:51:00] "PUT /update/653ac22b897acf0c40887a22 HTTP/1.1" 200 -
